In [1]:
import os, sys, gc
from pathlib import Path
from glob import glob
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
Data_dir = Path("../data/")
os.listdir(Data_dir)

['test_data.xlsx']

In [3]:
data = pd.read_excel(Data_dir/'test_data.xlsx')
data

,类目,标题
0,Pet Hair Trimmer,4 in 1 Pet Hair Clipper With 4 Blades Grooming...
1,Pet Hair Trimmer,"Animal Clipper pet care series sonar sn-270, r..."
2,Pet Hair Trimmer,Babyliss 35007690 block knife set (40mm) Clipp...
3,Pet Hair Trimmer,Dog Hair Trimmer USB Rechargeable Professional...
4,Pet Hair Trimmer,Dropshipping Dog Noise-Low Design Pet Hair Cli...
...,...,...
99998,Home Appliances,Multi-purpose Handheld Electric High-pressure ...
99999,Home Appliances,Multi-set Dirt Disposal Replacement Bags for i...
100000,Home Appliances,Multicooker Rice Cooker 11 in 1 DIY Functions ...
100001,Home Appliances,Multifunction Automatic UV Sterilizer for Toot...


In [4]:
data.rename(columns={'类目': 'Category', '标题': 'Description'}, inplace=True)
data

,Category,Description
0,Pet Hair Trimmer,4 in 1 Pet Hair Clipper With 4 Blades Grooming...
1,Pet Hair Trimmer,"Animal Clipper pet care series sonar sn-270, r..."
2,Pet Hair Trimmer,Babyliss 35007690 block knife set (40mm) Clipp...
3,Pet Hair Trimmer,Dog Hair Trimmer USB Rechargeable Professional...
4,Pet Hair Trimmer,Dropshipping Dog Noise-Low Design Pet Hair Cli...
...,...,...
99998,Home Appliances,Multi-purpose Handheld Electric High-pressure ...
99999,Home Appliances,Multi-set Dirt Disposal Replacement Bags for i...
100000,Home Appliances,Multicooker Rice Cooker 11 in 1 DIY Functions ...
100001,Home Appliances,Multifunction Automatic UV Sterilizer for Toot...


In [5]:
data.Category = data.Category.astype('str')
data.Description = data.Category.astype('str')

In [6]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
data['label'] = encoder.fit_transform(data['Category'])

In [7]:
from sklearn import model_selection

In [8]:
def create_folds(data, num_splits):
    # we create a new column called Kfold and fill it with -1
    data['kfold'] = -1
    
    # the next step is to randomize the rows of the data
    data = data.sample(frac=1).reset_index(drop=True)
    
    # calculate number of bins by Sturge's rule of 
    # Take the floor of the values or just round it up
    num_bins = int(np.floor(1 + np.log2(len(data))))
    
    # bin targets or simply the Category as shown in my data
    data.loc[:, "bins"] = pd.cut(
        data['label'], bins=num_bins, labels=False
    )
    
    # initiate the kfold class from model_selection module
    kf = model_selection.StratifiedKFold(n_splits=num_splits)
    
    # fill the new kfold column 
    # note that instead of targets, we use bins
    for fold, (tr_idx, val_idx) in enumerate(tqdm(kf.split(X=data, y=data.bins.values), total=num_splits, desc='Creating folds ...')):
        data.loc[val_idx, 'kfold'] = fold
        
    # drop the bins columns
    data = data.drop("bins", axis=1)
    
    # return the dataframe with folds
    return data

In [62]:
df = create_folds(data, 5)

Creating folds ...:   0%|          | 0/5 [00:00<?, ?it/s]

In [13]:
df.kfold.value_counts()

0    20001
1    20001
2    20001
3    20000
4    20000
Name: kfold, dtype: int64

# GROUPBY PRACTICE

In [51]:
df = pd.DataFrame(data = {'id': ['aaa', 'aaa', 'bbb', 'bbb', 'ccc'],
                          'val': ['hello', 'world', 'foo', 'bar', 'rabbit']})

In [52]:
df

,id,val
0,aaa,hello
1,aaa,world
2,bbb,foo
3,bbb,bar
4,ccc,rabbit


In [54]:
df.groupby(['id'], as_index=False).agg({'val': [' '.join, 'count']})

id          val      
               join count
0  aaa  hello world     2
1  bbb      foo bar     2
2  ccc       rabbit     1

In [98]:
count_desc = df.groupby(['Description'], as_index=False).agg({'Category': [' '.join, 'count']})
count_desc = pd.merge(count_desc, df, on=['Description'], how='left')

In [99]:
count_desc.columns

Index([        'Description',   ('Description', ''),  ('Category', 'join'),
       ('Category', 'count'),            'Category',               'label',
                     'kfold'],
      dtype='object')

In [104]:
df = pd.DataFrame({'store': ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B'],
                   'quarter': [1, 1, 2, 2, 1, 1, 2, 2],
                   'employee': ['Andy', 'Bob', 'Chad', 'Diane',
                                'Elana', 'Frank', 'George', 'Hank']})

#view DataFrame
print(df)

  store  quarter employee
0     A        1     Andy
1     A        1      Bob
2     A        2     Chad
3     A        2    Diane
4     B        1    Elana
5     B        1    Frank
6     B        2   George
7     B        2     Hank


In [112]:
gr_df = df.groupby(['store', 'quarter'], as_index=False).agg({'employee': ' & '.join})
gr_df

,store,quarter,employee
0,A,1,Andy & Bob
1,A,2,Chad & Diane
2,B,1,Elana & Frank
3,B,2,George & Hank


In [113]:
pd.merge(gr_df, df, on=['store', 'quarter'], how='left')

,store,quarter,employee_x,employee_y
0,A,1,Andy & Bob,Andy
1,A,1,Andy & Bob,Bob
2,A,2,Chad & Diane,Chad
3,A,2,Chad & Diane,Diane
4,B,1,Elana & Frank,Elana
5,B,1,Elana & Frank,Frank
6,B,2,George & Hank,George
7,B,2,George & Hank,Hank


In [114]:
df = pd.DataFrame({'team': ['A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B'],
                   'points': [12, 29, 34, 14, 10, 11, 7, 36, 34, 22]})

In [119]:
df.groupby('team')['points'].nlargest(3)

team   
A     2    34
      1    29
      3    14
B     7    36
      8    34
      9    22
Name: points, dtype: int64

In [120]:
df.groupby('team')['points'].nsmallest(3)

team   
A     4    10
      0    12
      3    14
B     6     7
      5    11
      9    22
Name: points, dtype: int64

In [121]:
df.groupby('team')['points'].apply(lambda grp: grp.nlargest(2).sum())

team
A    63
B    70
Name: points, dtype: int64

In [122]:
#create DataFrame
df = pd.DataFrame({'team': ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
                            'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B'],
                   'points': [4, 7, 7, 11, 12, 15, 19, 19,
                             5, 5, 11, 12, 14, 14, 15, 15]})

In [124]:
bins = int(np.floor(1+np.log2(len(df))))
bins

5

In [128]:
df.groupby(['team', pd.cut(df.points, bins=bins)]).size().unstack()

points,"(3.985, 7.0]","(7.0, 10.0]","(10.0, 13.0]","(13.0, 16.0]","(16.0, 19.0]"
team,,,,,
A,3,0,2,1,2
B,2,0,2,4,0


# SPARK BY EXAMPLE

In [129]:
technologies   = ({
    'Courses':["Spark","PySpark","Hadoop","Python","Pandas","Hadoop","Spark","Python","NA"],
    'Fee' :[22000,25000,23000,24000,26000,25000,25000,22000,1500],
    'Duration':['30days','50days','55days','40days','60days','35days','30days','50days','40days'],
    'Discount':[1000,2300,1000,1200,2500,None,1400,1600,0]
          })
df = pd.DataFrame(technologies)
print(df)

   Courses    Fee Duration  Discount
0    Spark  22000   30days    1000.0
1  PySpark  25000   50days    2300.0
2   Hadoop  23000   55days    1000.0
3   Python  24000   40days    1200.0
4   Pandas  26000   60days    2500.0
5   Hadoop  25000   35days       NaN
6    Spark  25000   30days    1400.0
7   Python  22000   50days    1600.0
8       NA   1500   40days       0.0


In [141]:
df.groupby(by=['Courses'], dropna=True).sum()

,Fee,Discount
Courses,,
Hadoop,48000,1000.0
NA,1500,0.0
Pandas,26000,2500.0
PySpark,25000,2300.0
Python,46000,2800.0
Spark,47000,2400.0


In [144]:
#create DataFrame
df = pd.DataFrame({'team': ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B'],
                   'points': [18, 22, 19, 14, 14, 11, 20, 28],
                   'assists': [2, 7, 9, 3, 12, 10, 14, 21]})

In [146]:
df.groupby(['team'])[['points', 'assists']].corr().unstack().iloc[:, 1]

team
A    0.603053
B    0.981798
Name: (points, assists), dtype: float64

In [149]:
df.groupby(['team'])[['points', 'assists']].corr().unstack().iloc[:, 1]

team
A    0.603053
B    0.981798
Name: (points, assists), dtype: float64